In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url
from torchvision.transforms import ToTensor
from torchvision import transforms
from torch.utils.data import random_split, ConcatDataset
import numpy as np
from PIL import Image
import tarfile
import os

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset = load_dataset("zh-plus/tiny-imagenet")

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
    valid: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})


In [ ]:
train_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4802, 0.4481, 0.3975], [0.2302, 0.2265, 0.2262]),
    ]
)

In [ ]:
class TinyImageNetDataset(Dataset):
    def __init__(self, dataset_split, transform=None):
        self.data = dataset_split
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]['image'].convert('RGB')
        label = self.data[idx]['label']
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_data = TinyImageNetDataset(dataset['train'], train_transforms)
val_data = TinyImageNetDataset(dataset['valid'], val_transforms)

train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=200):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(256 * 8 * 8, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes=200).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [ ]:
def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100. * correct / total
    return running_loss / len(loader), train_accuracy

In [ ]:
def evaluate(model, loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100. * correct / total
    return val_loss / len(loader), accuracy

In [ ]:
num_epochs = 30
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch 1/30, Train Loss: 4.7637, Val Loss: 4.1076, Val Accuracy: 12.58%
Epoch 2/30, Train Loss: 4.0382, Val Loss: 3.7564, Val Accuracy: 17.86%
Epoch 3/30, Train Loss: 3.7625, Val Loss: 3.5173, Val Accuracy: 21.23%
Epoch 4/30, Train Loss: 3.6183, Val Loss: 3.4079, Val Accuracy: 23.59%
Epoch 5/30, Train Loss: 3.5197, Val Loss: 3.3158, Val Accuracy: 24.71%
Epoch 6/30, Train Loss: 3.4474, Val Loss: 3.2919, Val Accuracy: 25.92%
Epoch 7/30, Train Loss: 3.3988, Val Loss: 3.2401, Val Accuracy: 25.96%
Epoch 8/30, Train Loss: 3.3438, Val Loss: 3.2691, Val Accuracy: 26.00%
Epoch 9/30, Train Loss: 3.3042, Val Loss: 3.2369, Val Accuracy: 26.35%
Epoch 10/30, Train Loss: 3.2660, Val Loss: 3.1801, Val Accuracy: 27.94%
Epoch 11/30, Train Loss: 3.2361, Val Loss: 3.1913, Val Accuracy: 27.45%
Epoch 12/30, Train Loss: 3.2125, Val Loss: 3.1527, Val Accuracy: 28.21%
Epoch 13/30, Train Loss: 3.1847, Val Loss: 3.1340, Val Accuracy: 28.46%
Epoch 14/30, Train Loss: 3.1665, Val Loss: 3.1140, Val Accuracy: 29.21%
E

In [ ]:
!pip install hiddenlayer

In [ ]:
!pip install torchsummary
from torchsummary import summary

In [ ]:
num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_accuracy = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

Epoch 1/20, Train Loss: 4.5879, Train Accuracy: 7.23%, Val Loss: 3.9378, Val Accuracy: 15.10%
Epoch 2/20, Train Loss: 3.8666, Train Accuracy: 16.14%, Val Loss: 3.5229, Val Accuracy: 21.01%
Epoch 3/20, Train Loss: 3.5798, Train Accuracy: 20.58%, Val Loss: 3.3637, Val Accuracy: 24.24%
Epoch 4/20, Train Loss: 3.3801, Train Accuracy: 23.66%, Val Loss: 3.2962, Val Accuracy: 25.58%
Epoch 5/20, Train Loss: 3.2362, Train Accuracy: 26.03%, Val Loss: 3.2379, Val Accuracy: 26.67%
Epoch 6/20, Train Loss: 3.1085, Train Accuracy: 27.98%, Val Loss: 3.2096, Val Accuracy: 26.37%
Epoch 7/20, Train Loss: 2.9913, Train Accuracy: 30.04%, Val Loss: 3.2006, Val Accuracy: 26.92%
Epoch 8/20, Train Loss: 2.8755, Train Accuracy: 31.80%, Val Loss: 3.1858, Val Accuracy: 27.36%
Epoch 9/20, Train Loss: 2.7635, Train Accuracy: 33.84%, Val Loss: 3.1867, Val Accuracy: 27.36%
Epoch 10/20, Train Loss: 2.6658, Train Accuracy: 35.46%, Val Loss: 3.2463, Val Accuracy: 26.96%
Epoch 11/20, Train Loss: 2.5684, Train Accuracy: 3